### Acceso a drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Bibliotecas 

In [2]:
!sudo apt install tesseract-ocr
!pip3 install pytesseract
!cp drive/MyDrive/HackathonRIIAA2021/Data/spa.traineddata /usr/share/tesseract-ocr/4.00/tessdata/

!apt install enchant
!pip install pyenchant
!sudo apt-get install myspell-es

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 40 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 2s (2,920 kB/s)
debconf: unable to initi

In [3]:
import os 
import re
import string
import pytesseract as ocr
import cv2 as cv
import numpy as np
from tqdm import tqdm
from enchant.checker import SpellChecker
from difflib import SequenceMatcher

In [4]:
# cleanup text
def get_personslist(text):
    personslist=[]
    for sent in nltk.sent_tokenize(text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if isinstance(chunk, nltk.tree.Tree) and chunk.label() == 'PERSON':
                personslist.insert(0, (chunk.leaves()[0][0]))
    return list(set(personslist))


# using enchant.checker.SpellChecker, identify incorrect words
def identify_incorrect_words(text):
  rep = { '\n': ' ', '\\': ' ', '\"': '"', '-': ' ', '"': ' " ', 
        '"': ' " ', '"': ' " ', ',':' , ', '.':' . ', '!':' ! ', 
        '?':' ? ' , '*':' * ', 
        '(': ' ( ', ')': ' ) ', '=-\n':''}
  rep = dict((re.escape(k), v) for k, v in rep.items()) 
  pattern = re.compile("|".join(rep.keys()))
  text = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)
  # personslist = get_personslist(text)
  # print("PERSON LIST")
  # ignorewords = personslist + ["!", ",", ".", "\"", "?", '(', ')', '*', '`']
  ignorewords = ["!", ",", ".", "\"", "?", '(', ')', '*']
  spell = SpellChecker("es_MX")
  words = text.split()
  incorrectwords = [w for w in words if not spell.check(w) and w not in ignorewords and len(w) > 1]
  # print("Incorrect WOrds: ", len(incorrectwords))
  # using enchant.checker.SpellChecker, get suggested replacements
  suggestedwords = [spell.suggest(w) for w in incorrectwords]
  # print("Suggested WOrds: ", len(suggestedwords))
  # replace incorrect words with [MASK]
  for w in incorrectwords:
    # print(w)
    # print(text)
    text = text.replace(" " + w + " ", ' [MASK] ')
    # print(text)
  return text, suggestedwords

In [5]:
!pip install transformers
import torch
from transformers import BertTokenizer,BertForMaskedLM
print('Loading BERT tokenizer...')


     |████████████████████████████████| 2.6 MB 7.5 MB/s 
     |████████████████████████████████| 895 kB 45.9 MB/s 
     |████████████████████████████████| 3.3 MB 59.9 MB/s 
     |████████████████████████████████| 636 kB 62.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Loading BERT tokenizer...


In [6]:
def get_bert_predictions(text, suggestedwords):
  # Load, train and predict using pre-trained model
  tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')

  #Remove punctuations
  punctuations = string.punctuation
  punctuations = punctuations.replace('[', '')
  punctuations = punctuations.replace(']', '')
  for c in text:
    if c in punctuations:
        text = text.replace(c, "")
  print("Text:", text)
  text = '[CLS] ' + text + ' [SEP]'
  tokenized_text = tokenizer.tokenize(text)
  print("Tokenized Text", tokenized_text)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  MASKIDS = [i for i, e in enumerate(tokenized_text) if e == '[MASK]']
  # Create the segments tensors
  segs = [i for i, e in enumerate(tokenized_text) if e == "."]
  segments_ids=[]
  prev=-1
  for k, s in enumerate(segs):
      segments_ids = segments_ids + [k] * (s-prev)
      prev=s
  segments_ids = segments_ids + [len(segs)] * (len(tokenized_text) - len(segments_ids))
  segments_tensors = torch.tensor([segments_ids])
  # prepare Torch inputs 
  tokens_tensor = torch.tensor([indexed_tokens])
  # Load pre-trained model
  model = BertForMaskedLM.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
  model.eval()
  
  # Predict all tokens
  with torch.no_grad():
      predictions = model(tokens_tensor, segments_tensors)
  pred_words=[]
  for i in range(len(MASKIDS)):
      # print("Obtener Predicciones i= {}, len = {}, MASKIDS[i]={}, len predictions= {}".format(i, len(MASKIDS), MASKIDS[i], len(predictions[0][0])))
      preds = torch.topk(predictions[0][0][MASKIDS[i]], k=5) 
      # print("tengo las predicciones")
      indices = preds.indices.tolist()
      # print("tengo los indices")
      # print(indices)
      list1 = tokenizer.convert_ids_to_tokens(indices)
      # print("tengo los tokens")
      # print(len(suggestedwords))

      list2 = suggestedwords[i]
      # print("Sugerencias SpellChecker", list2)
      # print("Sugerencias BERT", list1)
      simmax=0
      predicted_token=''
      for word1 in list1:
          for word2 in list2:
              # print("Word 1:{}, Word 2:{}".format(word1, word2))
              s = SequenceMatcher(None, word1, word2).ratio()
              if s is not None and s > simmax:
                  simmax = s
                  predicted_token = word1
      # print("predicted token:", predicted_token)
      text = text.replace('[MASK]', predicted_token, 1)
      # print("REPLACEEEEEEEEEEEEEEEE=================================================")
      # print(text)
  return text

### Función para procesar imagenes y utilizar Tessearct-OCR

In [8]:
def get_processed_images(src_path, processed_path, texts_path, bin_option=True, border_option=False):
  source = [f for f in os.listdir(src_path) if os.path.isfile(os.path.join(src_path, f))]
  print('\n[INFO] Se encontraron {} imagenes para procesar.\n'.format(len(source)))
  
  for index in tqdm(range(len(source))):
    # Lectura y escalado de imagenes #
    image_name = source[index].split('.')[0]
    image = cv.imread('{}{}'.format(src_path,source[index]))
    image = cv.resize(image, None, fx=1.3, fy=1.3, interpolation=cv.INTER_CUBIC)
    
    """
    Preprocesamiento de imagenes
    """
    # Eliminación de ruido causado por sombras en las imagenes #
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    dilated_image = cv.dilate(gray_image, np.ones((7,7), np.uint8))
    blur_image = cv.medianBlur(dilated_image,21)
    diff_image = 255 - cv.absdiff(gray_image, blur_image)
    norm_image = diff_image.copy()
    cv.normalize(diff_image, norm_image, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_8UC1)
    thr_img = cv.threshold(norm_image, 230, 0, cv.THRESH_TRUNC)[1]
    cv.normalize(thr_img, thr_img, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_8UC1)
    bin_image = cv.threshold(thr_img, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)[1]
    if border_option == False:
      # Lectura del texto de la imagen mediante OCR sin detección de bordes#
      config_options = r'--oem 1'
      text = ocr.image_to_string(bin_image, lang='spa', config=config_options)
      cv.imwrite('{}{}.png'.format(processed_path,image_name), bin_image)
      textfile = open('{}{}.txt'.format(texts_path,image_name),'w')
      textfile.write(text)
      textfile.close()
    else:
      # Lectura del texto de la imagen mediante OCR con detección de bordes#
      bin_image_inv = cv.threshold(thr_img, 0, 255, cv.THRESH_OTSU | cv.THRESH_BINARY_INV)[1]
      rect_kernel = cv.getStructuringElement(cv.MORPH_RECT, (90, 90))
      dilation = cv.dilate(bin_image_inv, rect_kernel, iterations = 1)
    
      contours, hierarchy = cv.findContours(dilation, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
      if bin_option:
        image_copy = bin_image.copy()
      else:
        image_copy = gray_image.copy()
      count = -1
      for cnt in contours:
        count +=1
        x,y,w,h = cv.boundingRect(cnt)
        cropped = image_copy[y:y + h, x:x + w]
        h,w = cropped.shape
        if h>1500 and w>2500:
          config_options = r'--oem 1'
          text = ocr.image_to_string(cropped, lang='spa', config=config_options)
          text, suggestedwords = identify_incorrect_words(text)
          text = get_bert_predictions(text, suggestedwords)
          if len(text) > 0:
            cv.imwrite('{}{}_{}.png'.format(processed_path,image_name,count), cropped)
            textfile = open('{}{}.txt'.format(texts_path,image_name),'a')
            textfile.write(text)
            textfile.close()

In [9]:
IMAGES_PATH = 'drive/MyDrive/Datos - Hackathon JusticIA/Fichas_auto/'
PROCESSED_PATH = 'drive/MyDrive/HackathonRIIAA2021/Processed_images/Fichas_auto_bert/'
TEXTS_PATH = 'drive/MyDrive/HackathonRIIAA2021/Texts/Fichas_auto_bert/'

In [10]:
get_processed_images(src_path=IMAGES_PATH, processed_path=PROCESSED_PATH, texts_path=TEXTS_PATH)


[INFO] Se encontraron 1000 imagenes para procesar.



100%|██████████| 1000/1000 [2:48:32<00:00, 10.11s/it]
